## Use states as voters instead of congressmen

In [2]:
import pandas as pd
import networkx as nx
import numpy as np
import re

import glob, os
import itertools
from tqdm import tqdm

import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
from shapely.geometry import LineString

import shapely

In [3]:
mmb = pd.read_csv('../dataset/HSall_members.csv')
pc = pd.read_csv('../dataset/HSall_parties.csv')

In [4]:
def create_members_df():
    temp_congress = mmb.groupby('icpsr', as_index=False)[['congress']].agg(lambda x: list(x))
    temp_party = mmb.groupby('icpsr', as_index=False)[['party_code']].agg(lambda x: list(set(x)))
    temp_congress = temp_congress.merge(temp_party)
    temp_congress['bioname'] = temp_congress['icpsr'].map(mmb[['icpsr', 'bioname']].set_index('icpsr').to_dict()['bioname'])
    temp_congress['state_abbrev'] = temp_congress['icpsr'].map(mmb[['icpsr', 'state_abbrev']].set_index('icpsr').to_dict()['state_abbrev'])
    party_codes_dic = pc[['party_name', 'party_code']].set_index('party_code').to_dict()['party_name']
    temp_congress['party_name'] = temp_congress['party_code'].apply(lambda x: [party_codes_dic[y] for y in x])
    return temp_congress

members = create_members_df()

_________________________________________________________________________________________________

In [5]:
members2state_dic = members.set_index('icpsr')['state_abbrev'].to_dict()
members_party_dict = members.set_index('icpsr')['party_name'].to_dict()

In [6]:
# try for congress 110
congress_110 = pd.read_csv('../dataset/download_votes_merged/congress_110.csv')

In [7]:
# add the column of state_abbrev translating the icpsr
congress_110['state_abbrev'] = congress_110['icpsr'].map(members2state_dic)

In [9]:
edgelist = pd.DataFrame()

for v in tqdm(set(congress_77['id'])):
    temp = congress_77[congress_77['id']==v]
    yy = temp[temp['vote']=='Yea']['icpsr']
    nn = temp[temp['vote']=='Nay']['icpsr']
    y = itertools.combinations(yy, 2)
    n = itertools.combinations(nn, 2)
    o = itertools.product(yy, nn)

    y = pd.DataFrame(y, columns = ['source', 'target'])
    y['weight'] = 1
    y['count'] = 1
    n = pd.DataFrame(n, columns = ['source', 'target'])
    n['weight'] = 1
    n['count'] = 1
    o = pd.DataFrame(o, columns = ['source', 'target'])
    o['weight'] = -1
    o['count'] = 1

    edgelist = pd.concat([edgelist, y, n, o])
    

edgelist = pd.concat([edgelist, pd.DataFrame({
        'source': edgelist['target'],
        'target': edgelist['source'],
        'weight': edgelist['weight'],
        'count': edgelist['count']})])

edgelist = edgelist.loc[edgelist['source'] < edgelist['target']]
edgelist = edgelist.groupby(['source', 'target', 'weight']).sum().reset_index()
edgelist = edgelist.groupby(['source', 'target', 'weight']).sum().reset_index()
edgelist['party'] = edgelist.apply(lambda row: 'in' if members_party_dict[row['source']] == members_party_dict[row['target']] else 'out', axis=1)
edgelist['state'] = edgelist.apply(lambda row: 'in' if members2state_dic[row['source']] == members2state_dic[row['target']] else 'out', axis=1)
map_votes = edgelist.groupby(['source', 'target'])['count'].sum().to_dict()
edgelist['votes_togheter'] = edgelist[['source', 'target']].apply(lambda x: map_votes[(x['source'], x['target'])], axis=1)
edgelist['perc'] = edgelist['count']/edgelist['votes_togheter']

SyntaxError: invalid syntax. Perhaps you forgot a comma? (3036106976.py, line 11)

In [ ]:
# translate the source and target to state_abbrev
edgelist['source_state'] = edgelist['source'].map(members2state_dic)
edgelist['target_state'] = edgelist['target'].map(members2state_dic)

# collapse the edgelist to state-based, the signed network should have +1 if more than 50% of the votes are in the same direction, -1 otherwise. report the percentage of votes in the same direction
edgelist_state = edgelist.groupby(['source_state', 'target_state', 'weight']).sum().reset_index()